# Regression Models with Keras

## Introduction


Despite the popularity of more powerful libraries such as PyToch and TensorFlow, they are not easy to use and have a steep learning curve. So, for people who are just starting to learn deep learning, there is no better library to use other than the Keras library. 

Keras is a high-level API for building deep learning models. It has gained favor for its ease of use and syntactic simplicity facilitating fast development. As you will see in this lab and the other labs in this course, building a very complex deep learning network can be achieved with Keras with only few lines of code. You will appreciate Keras even more, once you learn how to build deep models using PyTorch and TensorFlow in the other courses.

## Objectives for this Notebook    
* How to use the Keras library to build a regression model
* Download and clean the data set
* Build a neural network
* Train and test the network     



<h2>Table of Contents</h2>
1. <a href="##Download-and-Clean-the-Data-Set">Download and Clean the Data Set</a><br>
2. <a href="#Import-Keras-Packages">Import Keras Packages</a><br>
3. <a href="#Build-a-Neural-Network">Build a Neural Network</a><br>
4. <a href="#Train-and-Test-the-Network">Train and Test the Network</a><br>  


#### To use Keras, you will also need to install a backend framework – such as TensorFlow.

If you install TensorFlow 2.16 or above, it will install Keras by default.

We are using the CPU version of tensorflow since we are dealing with smaller datasets. 
You may install the GPU version of tensorflow on your machine to accelarate the processing of larger datasets


#### Suppress the tensorflow warning messages
We use the following code to  suppress the warning messages due to use of CPU architechture for tensoflow.

You may want to **comment out** these lines if you are using the GPU architechture


In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import pandas as pd
import numpy as np
import keras

import warnings
warnings.simplefilter('ignore', FutureWarning)

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

* Cement
* Blast furnace slag
* Fly ash
* Water
* Superplasticizer
* Coarse aggregate
* Fine aggregate


## Download and Clean the Data Set


Let's download the data and read it into a <em>pandas</em> dataframe.


In [3]:
filepath='https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have


In [4]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [7]:
concrete_data_columns = concrete_data.columns

In [8]:
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.


In [9]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [10]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [11]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [12]:
n_cols = predictors_norm.shape[1] # number of predictors

##  Import Keras Packages

##### Let's import the rest of the packages from the Keras library that we will need to build our regression model.


In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [14]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units.


## Train and Test the Network


Let's call the function now to create our model.


In [15]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.


In [16]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100
23/23 - 1s - 47ms/step - loss: 1668.6948 - val_loss: 1163.4348
Epoch 2/100
23/23 - 0s - 4ms/step - loss: 1544.2047 - val_loss: 1047.9375
Epoch 3/100
23/23 - 0s - 4ms/step - loss: 1359.0404 - val_loss: 876.7077
Epoch 4/100
23/23 - 0s - 4ms/step - loss: 1076.9199 - val_loss: 650.3996
Epoch 5/100
23/23 - 0s - 4ms/step - loss: 726.8630 - val_loss: 418.4374
Epoch 6/100
23/23 - 0s - 4ms/step - loss: 421.6600 - val_loss: 265.5717
Epoch 7/100
23/23 - 0s - 4ms/step - loss: 274.8254 - val_loss: 208.3725
Epoch 8/100
23/23 - 0s - 4ms/step - loss: 235.3302 - val_loss: 193.1805
Epoch 9/100
23/23 - 0s - 4ms/step - loss: 219.7586 - val_loss: 186.0988
Epoch 10/100
23/23 - 0s - 4ms/step - loss: 207.4664 - val_loss: 177.7474
Epoch 11/100
23/23 - 0s - 4ms/step - loss: 197.8173 - val_loss: 171.4041
Epoch 12/100
23/23 - 0s - 4ms/step - loss: 190.0582 - val_loss: 168.9416
Epoch 13/100
23/23 - 0s - 4ms/step - loss: 183.4454 - val_loss: 164.4112
Epoch 14/100
23/23 - 0s - 4ms/step - loss: 177.7587 -

<h3>Practice Exercise 1</h3>


Now using the same dateset,try to recreate regression model featuring five hidden layers, each with 50 nodes and ReLU activation functions, a single output layer, optimized using the Adam optimizer.


In [17]:
# Write your code here
def regression_model():
    input_colm = predictors_norm.shape[1]
    model= Sequential()
    model.add(Input(shape=(input_colm,)))  # Set the number of input features 
    model.add(Dense(50, activation='relu'))  
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu')) 
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))  
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

<h3>Practice Exercise 2</h3>


 Train and evaluate the model simultaneously using the fit() method by reserving 10% of the data for validation and training the model for 100 epochs


In [18]:
# Write your code here
model = regression_model()
model.fit(predictors_norm, target, validation_split=0.1, epochs=100, verbose=2)

Epoch 1/100
29/29 - 2s - 52ms/step - loss: 1511.0739 - val_loss: 958.9972
Epoch 2/100
29/29 - 0s - 4ms/step - loss: 622.6552 - val_loss: 214.6895
Epoch 3/100
29/29 - 0s - 4ms/step - loss: 238.8044 - val_loss: 187.4466
Epoch 4/100
29/29 - 0s - 4ms/step - loss: 201.4860 - val_loss: 173.1494
Epoch 5/100
29/29 - 0s - 4ms/step - loss: 180.5454 - val_loss: 155.3888
Epoch 6/100
29/29 - 0s - 4ms/step - loss: 160.8268 - val_loss: 142.2212
Epoch 7/100
29/29 - 0s - 4ms/step - loss: 143.0801 - val_loss: 125.9547
Epoch 8/100
29/29 - 0s - 4ms/step - loss: 126.9515 - val_loss: 109.0530
Epoch 9/100
29/29 - 0s - 4ms/step - loss: 113.0126 - val_loss: 99.3061
Epoch 10/100
29/29 - 0s - 4ms/step - loss: 96.2981 - val_loss: 82.8429
Epoch 11/100
29/29 - 0s - 4ms/step - loss: 85.2583 - val_loss: 72.9809
Epoch 12/100
29/29 - 0s - 4ms/step - loss: 76.8494 - val_loss: 64.8190
Epoch 13/100
29/29 - 0s - 4ms/step - loss: 66.0135 - val_loss: 58.0288
Epoch 14/100
29/29 - 0s - 4ms/step - loss: 60.1385 - val_loss: 51.6

Based on the results, we notice that:

- Adding more hidden layers to the model increases its capacity to learn and represent complex relationships within the data. This allows the model to better identify, as a result, the model becomes more effective at fitting the training data and potentially improving its predictions.
- By reducing the proportion of data set aside for validation and using a larger portion for training, the model has access to more examples to learn from. This additional training data helps the model improve its understanding of the underlying trends, which can lead to better overall performance.  


<!--
## Change Log

|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
| 2024-11-20  | 3.0  | Aman  |  Updated the library versions to current |
| 2020-09-21  | 2.0  | Srishti  |  Migrated Lab to Markdown and added to course repo in GitLab |



<hr>

## <h3 align="center"> © IBM Corporation. All rights reserved. <h3/>
